In [13]:
import numpy as np
import json_tricks

answer = {}


inputs = json_tricks.load('inputs/inputs.json')


# Matrix Product II

$$A_{5 \times 10} \  B_{10 \times 2}\ C_{2 \times 30}\ D_{30 \times 3}\ E_{3 \times 9}$$

1. What will be the shape of the resulting matrix?
2. How many multiplication of numbers are required at best?

In [4]:
answer['task1'] = {
    '1': (5, 9),
    '2': 424
}



# Numpy expression

Using Numpy, write a function that calculates the 
following expression:

$$\exp(A^T(B + 2C) + 3I) \mathbf x,$$

where $I$ is an identity matrix of the necessary shape.

In [8]:
def numpy_expression(A, B, C, x):
    res = x
    ## YOUR CODE HERE
    res_0 = np.dot(A.T, (B + 2*C))
    res_1 = res_0 + 3*np.eye(res_0.shape[0], res_0.shape[1])
    return np.exp(res_1).dot(x)

In [6]:
answer['task2'] = []
for one_input in inputs['task2']:
    answer['task2'].append(numpy_expression(**one_input))

# Einstein's Rule

In *Tensor Algebra*, a direct generalization of the Linear Algebra to the case of $N$-dimentional tables called *tensors* (normal matrix), the Einstein's rule exists.

It works as follows: if you see a duplicating upper and lower index in the formula, that means, this index convolves.

For example, the following tensor expression, summation and matrix product are equivalent:

$$a_k^l b_l^m = \sum_{l=1}^L a_k^l b_l^m = AB$$

In this notation subscript means row index and superscript means column index.

<details>
<summary> Note </summary>

> [!NOTE]
> Also at some point it will be important to know that:
> * lower index represents a contravariant dimension of a
> tensor
> * upper index represents a covariant dimension 
> of tensor. But let us omit this part for now.

</details>

# Task

Calculate the following expression written using Einstein's 
rule:

$$a_k^m b_m^n c_n^o d_l^k$$

In [9]:
def einsteins_rule(A, B, C, D):
    res = A
    ## YOUR CODE HERE
    res_1 = np.dot(np.dot(A,B), C)
    res_2 = np.dot(D, res_1)
    return res_2

In [10]:
answer['task3'] = []
for one_input in inputs['task3']:
    answer['task3'].append(einsteins_rule(**one_input))

# Diagonal Matrix Product

You are given two square matrices: $A$ and $D$, where $A$ is a 
full matrix and $D$ is a diagonal matrix:

$$
A = \begin{bmatrix}
- & \mathbf a_1 & - \\
& \vdots & \\
- & \mathbf a_N & - \\
\end{bmatrix}
$$

$$
D = \textrm{diag}(d_1, d_2, \dots, d_N) = \begin{bmatrix}
d_1 & & & & \\
& d_2 & & & \\
& & d_3 & & \\
& & & \ddots & \\
& & & & d_N 
\end{bmatrix}
$$

Write a program to calculate the result of $DA$ and $AD$ in 
the fastest possible way.

In [20]:
def diag_prod_DA(A, D):
    res = A
    ## YOUR CODE HERE
    D_diag_col = np.diag(D).reshape(-1, 1)
    print(D_diag_col)
    res = A * D_diag_col
    return res

def diag_prod_AD(A, D):
    res = A
    ## YOUR CODE HERE
    D_diag_row = np.diag(D).reshape(1, -1)
    res = D_diag_row * A
    return res

In [21]:
n = 3
D = np.diag([1, 2, 3])  # Диагональная матрица
A = np.random.rand(n, n)  # Полная матрица

DA = diag_prod_DA(A, D)
AD = diag_prod_AD(A, D)
print(DA, AD)

[[1]
 [2]
 [3]]
[[0.18111279 0.81990493 0.19941942]
 [1.89075759 0.66159768 1.12433949]
 [1.42805206 1.87420205 0.24734957]] [[0.18111279 1.63980985 0.59825825]
 [0.9453788  0.66159768 1.68650923]
 [0.47601735 1.24946803 0.24734957]]


In [12]:
answer['task4_1'] = []
answer['task4_2'] = []
for one_input in inputs['task4']:
    answer['task4_1'].append(diag_prod_DA(**one_input))
    answer['task4_2'].append(diag_prod_AD(**one_input))

# Sparse Matrix Product

You are given two matrices of the same shape: $A$ and $B$. Matrix $A$ is full
and is given in the form of `numpy.ndarray`.

The second matrix $B$ is **sparse**. That means that the 
majority of the items are equal to $0$ except for $M$. This matrix is given
as a set of non-zero elements of this matrix in form of $3 \times M$ `numpy.ndarray` as row-column-value tuple (COO sparse matrix form):

$$
\begin{bmatrix}
r_1 & c_1 & v_1 \\
r_2 & c_2 & v_2 \\
& \vdots & \\
r_M & c_M & v_M \\
\end{bmatrix}
$$

If in this struct two items correspond to the same location, consider the latter is correct.

Write the most efficient program that calculates $AB$.

Also return the ratio between the number of multiplication operations that are needed to calculate the sparse product and the number of operations for full product.

In [ ]:
def sparse_matrix_product(A, B_sparse):
    print('A', A)
    print('B_sparse', B_sparse)
    res = A
    ratio = 0

    ## YOUR CODE HERE
    sparse_rows, sparse_cols, sparse_data = B_sparse
    row = np.asarray(sparse_rows)
    col = np.asarray(sparse_cols)

    # # Проверка отрицательных индексов
    # if np.any(row < 0) or np.any(col < 0):
    #     raise ValueError("Индексы должны быть неотрицательными")

    # print(row)
    # print(col)
    # print(B_sparse)

    sparse_shape = (row.max() + 1 if len(row) > 0 else 0, 
                   col.max() + 1 if len(col) > 0 else 0)
    k_sparse, n = sparse_shape
    m, k_dense = A.shape

    # if k_dense != k_sparse:
    #     raise ValueError(f"Несовместимые размеры: A.shape={A.shape}, B.shape={sparse_shape}")

    res = np.zeros((m, n))
    nnz = len(sparse_data)
    for idx in range(nnz):
        val = sparse_data[idx]
        i = row[idx]
        j = col[idx]

        # # Проверка границ индексов
        # if i >= k_sparse:
        #     raise ValueError(f"Индекс строки {i} >= {k_sparse} (число строк в B)")
        # if j >= n:
        #     raise ValueError(f"Индекс столбца {j} >= {n} (число столбцов в B)")

        res[:, j] += A[:, i] * val

    full_ops = m * k_dense * n
    sparse_ops = m * nnz
    ratio = sparse_ops / full_ops # if full_ops != 0 else 0.0

    return res, ratio


In [32]:
answer['task5'] = []
for one_input in inputs['task5']:
    answer['task5'].append(sparse_matrix_product(**one_input))

A [[ 2 -1]
 [ 0  0]]
B_sparse [[ 0  0  0  0]
 [ 0  0  0  0]
 [-2  0 -3  0]]
A [[-3  2]
 [ 1  1]]
B_sparse [[ 0  0]
 [ 1  1]
 [-3  0]]
A [[ 1  2]
 [-1 -3]]
B_sparse [[1]
 [0]
 [2]]
A [[-1 -2]
 [ 0 -3]]
B_sparse [[ 1  0  0]
 [ 0  0  1]
 [-3  0  1]]
A [[-3 -1]
 [ 2 -2]]
B_sparse [[1]
 [1]
 [2]]
A [[-2  2]
 [-2 -1]]
B_sparse [[ 1  1  1]
 [ 1  0  1]
 [ 0 -3 -2]]
A [[-3  0]
 [-3  2]]
B_sparse [[ 1]
 [ 1]
 [-1]]
A [[ 2 -1]
 [ 0 -3]]
B_sparse [[ 1]
 [ 0]
 [-3]]
A [[ 0 -2]
 [-3  0]]
B_sparse [[1]
 [0]
 [2]]
A [[ 0  1]
 [-2  0]]
B_sparse [[ 1  0]
 [ 0  1]
 [-1  0]]


In [11]:
json_tricks.dump(answer, '.answer.json')

'{"task1": {"1": "5x9", "2": 424}, "task2": [{"__ndarray__": [[7.0], [-6.0]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[-5.0], [-24.0]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[17.0], [-57.0]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[-2.0], [-8.0]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[48.0], [-16.0]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[-9.0], [0.0]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[-19.0], [10.0]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[-28.0], [-27.0]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[30.0], [16.0]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[-16.0], [-2.0]], "dtype": "float64", "shape": [2, 1], "Corder": true}], "task3": [{"__ndarray__": [[24, 8], [6, 38]], "dtype": "int64", "shap